In [1]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy import geocoders

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Wikipedia URL

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Requesting and Getting response using the URL

In [3]:
s = requests.Session()
response = s.get(URL, timeout=10)

Using HTML Parser to parse through the page's source code

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')
pretty_soup = soup.prettify()

Using the correct Class id to scrape the table

In [5]:
table_ = soup.find('table', {"class":'wikitable sortable'})

Extracting each row from the HTML file and storing it in the list

In [6]:
list_row = []
for row in table_.findAll("tr"):
    list_row.append(row)

Creating a Dataframe to store the table values

In [7]:
data = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])

Filtering out only the table values and storing it in the DataFrame

In [8]:
for i in range(1, len(list_row)):
    data.loc[i, 'Postal Code'] = str(list_row[i]).split('\n')[1].split('>')[1]
    data.loc[i, 'Borough'] = str(list_row[i]).split('\n')[3].split('>')[1]
    data.loc[i, 'Neighbourhood'] = str(list_row[i]).split('\n')[5].split('>')[1]

Dropping the rows with "Borough" value "Not assigned"

In [9]:
data.drop(data.loc[data['Borough']=='Not assigned'].index, inplace=True)
data.reset_index(inplace = True)
data.drop(columns = 'index', axis = 1, inplace = True)

Checking the dataframe's dimensions

In [10]:
data.shape

(103, 3)

Reading Latitude and Longitude data from the CSV File provided

In [11]:
geo_data=pd.read_csv('http://cocl.us/Geospatial_data')

Viewing the Data

In [12]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging Postal code dataset and Latitude Longitude dataset

In [13]:
merge = pd.merge(data, geo_data, on='Postal Code')

After merging the DataFrames

In [14]:
merge.head(n = 10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
